In [ ]:
import pandas as pd 
import numpy as np
from pandas import DataFrame

In [3]:
# 데이터 로딩하기
# n_ay1합쳐놓은 파일에서 1열을 삭제하고 로딩함 = 1열 무의미해서...
path = "C:/Users/USER/Desktop/haley/빅콘테스트/train_test"
train = pd.read_csv(path+"/train_activity_label.csv",engine="python")
test = pd.read_csv(path+"/test_activity.csv",engine="python")
train.shape, test.shape #train,test의 행과열 감상 

((440323, 39), (175631, 38))

In [5]:
train.columns # 데이터 살펴보기

Index(['wk', 'acc_id', 'cnt_dt', 'play_time', 'npc_exp', 'npc_hongmun',
       'quest_exp', 'quest_hongmun', 'item_hongmun', 'game_combat_time',
       'get_money', 'duel_cnt', 'duel_win', 'partybattle_cnt',
       'partybattle_win', 'cnt_enter_inzone_solo', 'cnt_enter_inzone_light',
       'cnt_enter_inzone_skilled', 'cnt_enter_inzone_normal', 'cnt_enter_raid',
       'cnt_enter_raid_light', 'cnt_enter_bam', 'cnt_clear_inzone_solo',
       'cnt_clear_inzone_light', 'cnt_clear_inzone_skilled',
       'cnt_clear_inzone_normal', 'cnt_clear_raid', 'cnt_clear_raid_light',
       'cnt_clear_bam', 'normal_chat', 'whisper_chat', 'district_chat',
       'party_chat', 'guild_chat', 'faction_chat', 'cnt_use_buffitem',
       'gathering_cnt', 'making_cnt', 'label'],
      dtype='object')

In [5]:
test.columns # 데이터 살펴보기 

Index(['wk', 'acc_id', 'cnt_dt', 'play_time', 'npc_exp', 'npc_hongmun',
       'quest_exp', 'quest_hongmun', 'item_hongmun', 'game_combat_time',
       'get_money', 'duel_cnt', 'duel_win', 'partybattle_cnt',
       'partybattle_win', 'cnt_enter_inzone_solo', 'cnt_enter_inzone_light',
       'cnt_enter_inzone_skilled', 'cnt_enter_inzone_normal', 'cnt_enter_raid',
       'cnt_enter_raid_light', 'cnt_enter_bam', 'cnt_clear_inzone_solo',
       'cnt_clear_inzone_light', 'cnt_clear_inzone_skilled',
       'cnt_clear_inzone_normal', 'cnt_clear_raid', 'cnt_clear_raid_light',
       'cnt_clear_bam', 'normal_chat', 'whisper_chat', 'district_chat',
       'party_chat', 'guild_chat', 'faction_chat', 'cnt_use_buffitem',
       'gathering_cnt', 'making_cnt'],
      dtype='object')

In [6]:
# train, test셋의 ID는 index로 만든 뒤 컬럼에선 제거합니다.
train.index = train.acc_id
test.index = test.acc_id
train.drop("acc_id", axis=1, inplace=True)
test.drop("acc_id", axis=1, inplace=True)

In [7]:
train['label'].value_counts() # label이 어떻게 생겼는지 감상 

retained    167110
2month       98642
month        97473
week         77098
Name: label, dtype: int64

# one-hot-encoding
데이터에는 수치형 데이터와 텍스트 데이터나 범주형 데이터가 있다. 
머신러닝이나 딥러닝 알고리즘은 수치로 된 데이터만 이해할 수 있다.
그래서 기계가 이해할 수 있는 형태로 데이터를 변환해 주어야 하는데 범주형 데이터는 원핫인코딩 형태로 변환해 준다.
원핫인코딩이란 해당되는 하나의 데이터만 1로 변경해 주고 나머지는 0으로 채워주는 것을 뜻한다.

one-hot-encoding-1은 4가지 이탈기준을 01로 구분한것이고
one-hot-encoding-2는 이탈과 비이탈기준으로 01을 구분한것이다. 
분석력이 더 좋게 위해서 one-hot-encoding-2 를 사용하여 돌리기로 한다. 
BUT! 현재 코드는 0,1의 비율이 맞지 않는데 (=한쪽으로 너무 쏠림)
샘플 추출시 이거 해결하는 코드가 있다고 함 

In [7]:
## one-hor-encoding-1
## pd.get_dummies로 One-hot 인코딩
#label_one_hot_encoded = pd.get_dummies(train.label) 

## 원래 데이터와 비교식으로 보여주기용 데이터프레임
#label_with_onehot = pd.concat(
#	[DataFrame(train.label), label_one_hot_encoded],
#	axis=1) 
#label_with_onehot.head(10)

In [8]:
#one-hot-encoding-2
train.loc[train["label"] == "retained", "label"] = 0
train.loc[train["label"] == "2month", "label"] = 1
train.loc[train["label"] == "month", "label"] = 1
train.loc[train["label"] == "week", "label"] = 1
train['label'].value_counts() #0과 1의 갯수 확인 

1    273213
0    167110
Name: label, dtype: int64

##인덱스처리 이유.... 궁금하다 

인덱스 말그대로 목차를 말하는 것이다.
데이터베이스 즉 데이터를 저장하는 장소인데 우리가 데이터를 막 저장했는데 
왜 저장할까? 바로 거기서 찾아서 쓸려고 하는 것이다.
즉 정보검색을 위해서 우리는 여러가지 방법이 있는 그냥 순서대로 일일히 비교해서 검색하는 방법과
두번쨰로 색인 즉 인덱스를 남겨서 그걸 비교하면서 찾는 방법이 있다.
어느 것이 더 빠를 것인가? 당연히 인덱스를 남겨서 찾는 부분이 더 빠르다.

In [9]:
#그리고 train셋의 TARGET 컬럼은 별도로 분류합니다.
# TARGET컬럼 만들기 #label 타겟처리
y = train.label
train = train.iloc[:,:-1]

train.shape, test.shape #train,test의 행과열 감상 

((440323, 37), (175631, 37))

In [10]:
#### 여기 잘 이해 안감 ㅜ 
# 데이터를 간단히 살펴보도록 하겠습니다.
# 우선 데이터 타입먼저 확인해보겠습니다. 모두 수치형 데이터인 것을 알 수 있습니다.
# 별도의 Ont-Hot-Encoding은 필요하지 않은 것으로 보입니다.
# (물론 수치형 데이터이지만 명목형 변수와 다름없어서 One-Hot-Encode해야하는 경우도 있겠지만
# 컬럼의 의미를 모르는 상태에서는 확인이 어렵습니다. 우선 모두 수치로 처리합니다)
types = DataFrame(train.dtypes).reset_index()
types.columns = ["Features","Types"]
types.groupby("Types").size()

Types
int64       2
float64    35
dtype: int64

In [11]:
#### 여기는 이거 실행하면 안되는것 아닌가 싶은데 실행시켜도 변화 없음 ㅋㅋㅋ
#혹시 모를 경우를 대비한 데이터 전처리를 수행합니다.
#모든 컬럼값이 동일한 케이스, 즉 실질적으로 constant인 컬럼이 있을 수 있습니다.
#해당 컬럼들을 확인, 미리 제거합니다.

train = train.loc[:,train.apply(lambda x:len(x.unique()))>1]
test = test.loc[:,test.apply(lambda x:len(x.unique()))>1]
train.shape, test.shape #train,test의 행과열 감상 

((440323, 37), (175631, 37))

constant인 컬럼 이 뭘 까? 

In [12]:
#제거한 결과, train셋에서는 constant가 아니었지만 
#(=1개 이상의 값이 있었지만) test 셋에서는 constant인 케이스가 있습니다. 
#이 결과 test셋에서 11개의 feature가 더 제거된 것을 알 수 있습니다. 
#test 셋에서 constant라면 train셋에서 학습하는 것이 의미가 없습니다. 
#train 셋의 컬럼을 test의 컬럼과 동일하게 만들어줍니다.

# test컬럼에는 없지만 train에만 있는 컬럼 제거하기 -> 전처리임 
testCols = test.columns
train = train[testCols]
train.shape, test.shape

((440323, 37), (175631, 37))

# 튜닝 설명 
#n_estimators
 - 몇번이나 모델링을 할 것인지 주기를 결정한다.
 - underfitting- traing data와 new data에 대해 예측이 떨어짐
 - overfitting- traing data에 대해서는 정확하지만 새 데이터에 대해 오히려 부정확한 결과를 낳음.
 - 일반적으로 100-1000이 적합한 learning rate다.

#learning_rate
 - 이전트리의 오차를 보정하는 정도를 조절 
 - n_estimators와 매우 연관되며 learning_rate를 낮추면 비슷한 복잡도의 모델을 만들기 위해서더 많은 트리를 추가해야한다. n_estimators가 클수록 좋은 랜덤 포레스트와는 달리 그래디언트부스팅에서 n_estimators를 크게하면 모델이 복잡해지고 과대적합될 가능성이 높아진다. 
 - 일반적인 관례는 가용한 시간과 메모리 한도에서 n_estimators를 맞추고 나서 적절한 
learning_rate를 찾는 것이다. 

#max_depth [default=6] (Should be tuned using CV) 
 - 트리의 최대 깊이.
 - 일반적으로 3-10
 - 트리의 복잡도를 낮추는 명령어이다. 통상 그래디언트 부스팅 모델에서는 max_depth를 매우 작게 설정하며 트리의 깊이가 5보다 깊어지지 않게 한다.
    
#subsample [default=1]
   - 각 트리마다의 관측 데이터 샘플링 비율.
   - 값을 적게 주면 over-fitting을 방지하지만 값을 너무 작게 주면 under-fitting.
   - 일반적으로 0.5-1
    
#colsample_bytree [default=1]
   - 각 트리마다의 feature 샘플링 비율.
   - 일반적으로 0.5-1

#objective [default=reg:linear]
   - binary:logistic : 이진 분류를 위한 로지스틱 회귀, 예측된 확률을 반환한다. (not class)
   - multi:softmax : softmax를 사용한 다중 클래스 분류, 예측된 클래스를 반환한다. (not probabilities)
   - multi:softprob : softmax와 같지만 각 클래스에 대한 예상 확률을 반환한다.
        
#eval_metric [default according to objective]
   - 회귀 분석인 경우 'rmse'를, 클래스 분류 문제인 경우 'error'를 default로 사용.
   - rmse : root mean square error 제곱 평균 제곱근 오차
   - mae : mean absolute error 절대 오차
   - logloss : negative log-likelihood 음수 대수 (log-likelihood)
   - error : Binary classification error rate (0.5 threshold)이진 분류 오류율 (0.5 임계 값)
   - merror : Multiclass classification error rate 멀티 클래스 분류 오류율
   - mlogloss : Multiclass logloss 멀티 클래스 로그 손실
   - auc : Area under the curve 곡선 아래 영역

In [ ]:
# train, test 셋 나누기 

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train,y,test_size=0.3, 
                                                    random_state=1234)

# Feature Selection에 활용할 Xgboost 모형 만들기

import xgboost as xgb
import time

params = {"learning_rate" : 0.02,
                             "n_estimators" : 3000,
                             "max_depth" : 7,
                             "subsample" : 1,
                             "colsample_bytree" : 0.5,
                              #"tree_method" : 'gpu_hist',
                              #"predictor" : 'gpu_predictor',
                               "n_jobs" : 12,
                             "objective" : "binary:logistic"}


clf = xgb.XGBClassifier(**params)

startTime = time.time()
clf.fit(X_train, y_train,
                    eval_set = [(X_train,y_train), (X_test,y_test)],
                    eval_metric='auc',
                    early_stopping_rounds=100,
                    verbose=200)
print("Spent", time.time() - startTime, "sec")

NameError: name 'train' is not defined

In [63]:
Imps = DataFrame({"Features":train.columns, 
                  "Imps":clf.feature_importances_}) \
                  .sort_values(by="Imps", ascending=False)
Imps.head(10)


,Features,Imps
2,play_time,0.168245
6,quest_hongmun,0.060753
8,game_combat_time,0.057564
9,get_money,0.053549
0,wk,0.049432
4,npc_hongmun,0.049126
7,item_hongmun,0.047400
34,cnt_use_buffitem,0.042923
29,whisper_chat,0.042515
31,party_chat,0.039535


In [64]:
# 변수 중요도가 0이 아닌 변수 목록을 list로 만들기
impVars = list(Imps[Imps.Imps != 0].iloc[:,0])

# 해당 변수로만 데이터셋 만들기
train = train[impVars]
test = test[impVars]

train.shape, test.shape

((440323, 37), (175631, 37))

In [65]:
## 성능 검증용 펑션 (roc_auc_score) 및 학습셋/검증셋 구분 위한 펑션 (train_test_split) 호출

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

## 학습셋/검증셋 구분
Xtrain, Xtest, ytrain,ytest = train_test_split(train,y,test_size=0.3)

## 파라미터 조합을 저장할 DataFrame 만들어두기
parameters = DataFrame({
                                         "3colsample":[0],
                                         "4max_depth" : [0],
                                         "5gamma" : [0],
                                         "6reg_l2" : [0],
                                         "7reg_l1" : [0],
                                         "8AUC":[0]})

## 검증할 파라미터 조합 List로 만들기
colsample_bytree = [0.4, 0.6, 0.8]
max_depth = [4, 8,12]
gamma = [0,2]
reg_l2 = [0,0.5]
reg_l1 = [0,0.5]

In [ ]:
i = 0

## 파라미터 조합별 반복 설정..
for colsample in colsample_bytree:
    for max in max_depth:
        for gam in gamma:
            for l2 in reg_l2:
                for l1 in reg_l1:
                    
                    ## 파라미터 설정
                    params = {"learning_rate" : 0.02,
                                             "n_estimators" : 3000,
                                             "max_depth" : max,
                                             "colsample_bytree" : colsample,
                                             "gamma" : gam,
                                              "lambda" : l2,
                                              'alpha' : l1,
                                             #"tree_method" : 'gpu_hist',
                                              #"predictor" : 'gpu_predictor',
                                               "n_jobs" : 14,
                                             "objective" : "binary:logistic"}
                    
                    clf = xgb.XGBClassifier(**params)
                        
                    s = time.time()  ## 적합 시작시간 저장
                    clf.fit(Xtrain, ytrain,
                                        eval_set = [(Xtrain,ytrain), (Xtest,ytest)],
                                        ## 적합중 early-stopping 하기위한 set 지정
                                        eval_metric='auc',
                                        ## early-stopping에 활용할 성능지표
                                        early_stopping_rounds=100,
                                        ## early-stopping시 100회 적합시 성능개선 없으면 학습 종료하도록 설정
                                        verbose=500
                                        ## 진행상황 500회마다 출력 
                                        )
                    print(time.time() - s)  ## 적합 소요시간 출력

                    r =  DataFrame({
                                                      "3colsample":[colsample],
                                                      "4max_depth" : [max],
                                                      "5gamma" : [gam],
                                                      "6reg_l2" : [l2],
                                                      "7reg_l1" : [l1],
                                                     "8AUC" : roc_auc_score(ytest,clf.predict_proba(Xtest)[:,1])
                                                       })
                    parameters = r.append(parameters)
                            
                    print(i, "finished")
                    i = i+1


[0]	validation_0-auc:0.787415	validation_1-auc:0.786327
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 100 rounds.
[500]	validation_0-auc:0.882562	validation_1-auc:0.87996
[1000]	validation_0-auc:0.89026	validation_1-auc:0.886734
[1500]	validation_0-auc:0.894862	validation_1-auc:0.890496


KeyboardInterrupt: 

# 테쓰또 F1 스코어하는방법 

In [15]:
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(train,y,random_state=5) #ㄱ

In [76]:
#X_eval = X_test
#y_eval = y_test

In [17]:
def xgb_model(X_train,X_eval,y_train,y_eval):
    xgb_train = xgb.DMatrix(X_train, label=y_train)
    xgb_eval = xgb.DMatrix(X_eval, y_eval) 
    
    xgb_params = {
        "objective"         : "binary:logistic" # loss function. binary:logistic (logistic regression for binary classification => predicted class) # multi:softmax (multicalss classification using the softmax objective -> predicted class , ) # multi:softprob: softmax와 같으나 class 대신 probability를 return
        #,"num_class"        : 2                # multi:softmax 쓸 경우, num_class (number of unique classes) 세팅해줘야함
        ,"eval_metric"      : "auc"              # validation에 사용될 metric (종류: rmse, mae, logloss, error, merror, mlogloss, auc)
        ,"eta"              : 0.1                # learning rate in GBM과 유사한 개념 (통상 0.01 ~ 0.2 사이 값)
        ,"max_depth"        : 6                  # maximum depth of a tree (overfitting 방지 위해 사용. depth 클수록 구체적 관계 학습. 통상 3 - 10) 
        ,"min_child_weight" : 10                 # minimum sum of weights of all observations (default=1, overfitting 방지를 위해 사용, 너무 높으면 underfitting 우려)
        ,"gamma"            : 0.70                # 분할에 필요한 최소 손실 함수, 이 이상으로 감소할 때만 분할됨 (default = 0)
        ,"subsample"        : 0.76                # 각 트리에 대해 임의의 샘플이되는 관측 값의 비율. 낮은 값이 보수적&overfitting을 막음. 너무 낮으면 underfitting 우려 (통상 0.5 - 1)
        ,"colsample_bytree" : 0.95                # 각 트리에 대해 임의의 샘플이 되는 columns의 비율. (default 1, 통상 0.5 - 1)
        ,"alpha"            : 2e-05               # L1 regularization term on weights (Lasso Regression) (default=0)
        ,"lambda"           : 1                   # L2 regularization term on weights.(Ridge Regression) (default=1)
    }

    watchlist  = [(xgb_eval,'eval'), (xgb_train,'train')]
    return model_training(xgb_params ,xgb_train ,watchlist)

def model_training(xgb_params ,xgb_train, watchlist):
    print('training XGBoost model ...')
    global xgb_model
    xgb_model = xgb.train(params = xgb_params,
                dtrain = xgb_train,
                num_boost_round = 300,
                evals = (watchlist),
                verbose_eval=10) 
    return xgb_model

In [18]:
xgb_model(X_train, X_test, y_train, y_test)

NameError: name 'xgb' is not defined

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
#import graphviz

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel 
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
def feature_importances(train_data,  test_data, train_target,test_target):
    rf = RandomForestClassifier(criterion='entropy', n_estimators=300, random_state=0,n_jobs=-1)
    rf.fit(train_data, train_target)
    rf.feature_importances_
    return selectModel(rf, train_data,  test_data, train_target,test_target)

In [3]:
def  selectModelselectMod (rf, train_data,  test_data, train_target,test_target):
    model = SelectFromModel(rf, prefit=True)
    feature_idx = model.get_support()
    feature_name = train_data.columns[feature_idx]
    train_data = model.transform(train_data)
    test_data = model.transform(test_data)
    global train_data_df
    train_data_df = pd.DataFrame(train_data, columns=feature_name)
    global test_data_df 
    test_data_df = pd.DataFrame(test_data, columns=feature_name)
    return gridsearch(train_data_df ,train_target, test_data_df , test_target)

In [98]:
def gridsearch(train_data ,train_target ,test_data, test_target):
    param = [{'n_estimators':[100,200,300,400],
         'criterion': ['gini', 'entropy']}]
    model = GridSearchCV(RandomForestClassifier(n_jobs=-1), param, cv=3)
    model.fit(train_data, train_target)
    print (model.best_params_)
    global pred_y
    pred_y = model.predict_proba(test_data)
    pred_y= pd.DataFrame(pred_y, columns=['n','y'])
    pred_y= pd.DataFrame(pred_y, columns=['n','y'])
    del pred_y['n']
    #print (pred_y)
    return f1score(pred_y,test_target)

In [4]:
def f1score(pred_y,test_target):
    i = 0.0
    f1_max = 0
    predic = 0
    while(i <= 0.5):
        pred = pred_y['y'].apply(lambda x: 1 if x > i else 0)
        f1 = f1_score(test_target, pred)
        if f1 > f1_max:
            predic = pred
            f1_max = f1
            acc = accuracy_score(test_target, pred)
            recall = recall_score(test_target, pred)
            threshold = i
        i += 0.001
    print ("threshold of max_f1score : {} ".format(threshold))
    print ("1) accuracy_score : {} ".format(acc))
    print ("2) recall_score : {} ".format(recall))
    print ("3) f1_score : {}".format(f1_max ))
    #return auc(test_target ,predic)

In [5]:
def auc(y,pred):
    print ("4) roc_auc : {}".format(roc_auc_score(y, pred)))
    print (classification_report(pred,y))
    return pred

In [6]:
1+3

4

In [101]:
a = xgb_model.get_score(importance_type='gain')
a = a.keys()
a = list(a)
len(a)

37

In [102]:
X_test = xgb.DMatrix(X_test)

TypeError: can not initialize DMatrix from DMatrix

In [103]:
pred_y = xgb_model.predict(X_test)

In [104]:
fpr_rf, tpr_rf, _ = roc_curve(y_test, pred_y)

In [105]:
pred_xgb= pd.DataFrame(pred_y ,columns=['y'])
f1_score(pred_xgb , y_test)

ValueError: Can't handle mix of continuous and unknown

In [41]:
#####################떄쓰또!2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, y,random_state=5)

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel 
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
import scikitplot.plotters as skplt
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

ModuleNotFoundError: No module named 'scikitplot'

In [34]:
hyperparameter_spacehyperpara  =  [{'n_estimators':[100, 300, 500, 1000],
         'criterion': ['gini', 'entropy']
         }]

In [35]:
rf = RandomForestClassifier(criterion='entropy', n_estimators=300, random_state=0,n_jobs=-1)
rf.fit(X_train, y_train)
rf.feature_importances_

ValueError: Unknown label type: 'unknown'

In [36]:
from sklearn.feature_selection import SelectFromModel 
from sklearn.metrics import f1_score

In [37]:

modelmodel  ==  SelectFromModelSelectFro (rf, prefit=True)
feature_idx = model.get_support()
feature_name = train_data.columns[feature_idx]
train_data = model.transform(train_data)
test_data = model.transform(test_data)

NameError: name 'modelmodel' is not defined